In [ ]:
#----------------------------IMPORTS----------------------------
import cv2
import numpy as np
import os
import random
import pickle
import tensorflow as tf
import keras 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#---------------------------READING DATASET LABELS----------------------

images_path = "D:\Work\CCE\Year 4 - Senior 2\Semester 1\Senior-2-Semester-1\GP\Graduation-Project\Clothes Description Module\Datasets\Dataset 1\Images"
csv_path = "D:\Work\CCE\Year 4 - Senior 2\Semester 1\Senior-2-Semester-1\GP\Graduation-Project\Clothes Description Module\Datasets\Dataset 1\CSV\images.csv"
df = pd.read_csv(csv_path) # Reading the csv file into a dataframe using pandas
df.head() # Displaying the first 5 rows of the dataframe

# # Removing all entries with label = Not sure or label Other
df = df[df['label'] != 'Not sure']
df = df[df['label'] != 'Other']

#Remove colums: sender_id and kids
df = df.drop(['sender_id', 'kids'], axis=1)
df.head() # Displaying the first 5 rows of the dataframe

# Extract Unique Labels = Number of classes but remove the 'Not sure' and 'Other' labels
class_names = df['label'].unique()
num_classes = len(class_names)
class_ids = dict(zip(class_names, range(num_classes))) # Make a dictionary of class names and their corresponding ids

print(f'There are {num_classes} classes: {class_names}')

In [ ]:
#---------------------------READING IMAGES AND THEIR LABELS----------------------
images = []
images_labels = []

#Read all the images in the file with the path images_path
for filename in os.listdir(images_path):
    image_path = os.path.join(images_path, filename)
    image = cv2.imread(image_path)
    filename = filename.split('.')[0]
    label = df.loc[df['image'] == filename]['label']
    if(label.empty or image is None):
        continue
    label = label.values[0]
    image = cv2.resize(image, (128, 128))
    images.append(image)
    images_labels.append(class_ids[label])
        
# Convert the images and labels to numpy arrays        
images = np.array(images, dtype=np.float32)
images_labels = np.array(images_labels, dtype=np.int32)

In [ ]:
#Normalize the images
images = images / 255.0

In [ ]:
#----------------------------------------DATA SPLITTING----------------------------------------

# Split the dataset into 70% for training and 15% for testing and 15% for cross-validation
trainval_images, x_test, trainval_labels, y_test = train_test_split(images, images_labels, test_size=0.15, random_state=42)

x_train, x_cv, y_train, y_cv = train_test_split(trainval_images, trainval_labels, test_size=0.15/0.85, random_state=42)

m_train = len(x_train)
m_test = len(x_test)
m_cv = len(x_cv)

# Print the size of each set
print(f"Dataset Size: {images.shape[0]}")
print(f"Training set size: {m_train} images")
print(f"Cross-validation set size: {m_cv} images")
print(f"Testing set size: {m_test} images")

In [ ]:
#Perform Data Augmentation on x_train and y_train using tensorflow ImageDataGenerator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Create a generator for the training set
train_generator = train_datagen.flow(x_train, y_train)

#use the train_generator
x_train, y_train = train_generator.next()

print(f"the shape of the training set (input) is: {len(x_train)}")
print(f"the shape of the training set (target) is: {len(y_train)}\n")

In [ ]:
#----------------------------------------BUILD MODEL USING TRANSFER LEARNING----------------------------------------

# Use ResNet pretrained model
# Load the pre-trained ResNet50 model without the top layer (i.e., the fully connected layers)
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained layers to avoid changing their weights during training
for layer in resnet_model.layers:
    layer.trainable = False

# Add a new fully connected layer for the specific classification task
x = Flatten()(resnet_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(num_classes, activation='linear')(x)

# Create a new model by combining the pre-trained ResNet50 model with the new fully connected layers
model = Model(inputs=resnet_model.input, outputs=x)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#----------------------------------------BUILD MODEL USING CNN----------------------------------------

# Create a convolutional neural network (CNN)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (128, 128, 3),kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Conv1'),
    tf.keras.layers.MaxPooling2D(2, 2, name = 'Max1'),
    tf.keras.layers.Conv2D(64, (5, 5), padding = 'same', activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Conv2'),
    tf.keras.layers.MaxPooling2D(2, 2, name = 'Max2'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Conv3'),
    tf.keras.layers.MaxPooling2D(2, 2, name = 'Max3'),
    tf.keras.layers.Conv2D(512, (5, 5), strides = 2, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Conv4'),
    tf.keras.layers.MaxPooling2D(2, 2, name = 'Max4'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Dense1'),    
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name = 'Dense2'),    
    tf.keras.layers.Dense(len(class_names), activation='linear', name = 'Dense3')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [ ]:
#-----------------------------------------------TRAINING----------------------------------------

# Define the EarlyStopping callback
# the training process will stop if the validation loss does not improve after 3 epochs.
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Define the ModelCheckpoint callback
# his callback can be used to save the best model weights during training based on a given metric (e.g., validation accuracy or loss).
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

history = model.fit(x_train, y_train, epochs=20, batch_size=30, validation_data=(x_cv, y_cv),callbacks=[early_stopping,checkpoint])

In [ ]:
#----------------------------------DELETE MODEL----------------------------------
del model

In [ ]:
#-----------------------------------SAVE MODEL-----------------------------------
pickle.dump(model, open('clothing_detector.pkl', 'wb'))

In [ ]:
#----------------------------------------PREDICTIONS----------------------------------------
train_predictions = model.predict(x_train)
cv_predictions = model.predict(x_cv)
test_predictions = model.predict(x_test)

train_count_correct = 0
cv_count_correct = 0
test_count_correct = 0

for i in range(m_train):
    predicted = np.argmax(train_predictions[i])
    if (predicted == y_train[i]):
        train_count_correct += 1
train_accuracy = train_count_correct / m_train

for i in range(m_test):
    predicted = np.argmax(test_predictions[i])
    if (predicted == y_test[i]):
        test_count_correct += 1
test_accuracy = test_count_correct / m_test

for i in range(m_cv):
    predicted = np.argmax(cv_predictions[i])
    if (predicted == y_cv[i]):
        cv_count_correct += 1
cv_accuracy = cv_count_correct / m_cv

print("Train Accuracy = %.2f" % (train_accuracy*100),'% with', m_train, 'training examples')
print("CV Accuracy = %.2f" % (cv_accuracy*100),'% with', m_cv, 'test examples')
print("Test Accuracy = %.2f" % (test_accuracy*100),'% with', m_test, 'test examples')
